#### Some of this code was taken from that of the booster and there may be some resemblance. ####
## WARNING: IT IS IMPORTANT THAT YOU RUN THE COMMENTED !pip install git blah blah blah COMMAND BELOW OR ELSE NOTHING WILL WORK. THIS IS A SPECIFIC VERSION/TYPE OF MINICONF THAT YOU SHOULD HAVE ON YOUR PC BECAUSE THERE ARE OTHERS THAT WON'T MAKE IT WORK!!! ##

In [111]:
import paho.mqtt.client as mqtt
# !pip install git+https://github.com/quartiq/miniconf@v0.16.0#subdirectory=py/miniconf-mqtt
import miniconf
import json
import pyvisa
import time
import numpy as np
import asyncio
from miniconf import Miniconf
import scipy
import sympy

In [112]:
# Configuration
broker = "localhost"            # replace with your broker IP or hostname
stabilizer_name = "80-34-28-5f-2c-e0" # check this name from the MQTT explorer window
prefix = f"dt/sinara/dual-iir/{stabilizer_name}"       # topic to subscribe to

In [113]:
# get a list of possible options
async with miniconf.Client(broker, protocol=miniconf.MQTTv5, ) as client:
# async is used in Python to do asynchronous programming -> allows tasks to run without blocking the entire program while waiting for 
# something to finish. It allows multiple tasks to run concurrently without waiting for one to finish before starting another.

    dev = miniconf.Miniconf(client, prefix)
    print("connected to miniconf")

    test = await dev.list()
    for option in test:
       print(option)

connected to miniconf
/ch/0/gain
/ch/0/biquad/0/typ
/ch/0/biquad/0/repr/Ba/ba
/ch/0/biquad/0/repr/Ba/u
/ch/0/biquad/0/repr/Ba/min
/ch/0/biquad/0/repr/Ba/max
/ch/0/biquad/0/repr/Raw
/ch/0/biquad/0/repr/Pid/order
/ch/0/biquad/0/repr/Pid/gain/i2
/ch/0/biquad/0/repr/Pid/gain/i
/ch/0/biquad/0/repr/Pid/gain/p
/ch/0/biquad/0/repr/Pid/gain/d
/ch/0/biquad/0/repr/Pid/gain/d2
/ch/0/biquad/0/repr/Pid/limit/i2
/ch/0/biquad/0/repr/Pid/limit/i
/ch/0/biquad/0/repr/Pid/limit/p
/ch/0/biquad/0/repr/Pid/limit/d
/ch/0/biquad/0/repr/Pid/limit/d2
/ch/0/biquad/0/repr/Pid/setpoint
/ch/0/biquad/0/repr/Pid/min
/ch/0/biquad/0/repr/Pid/max
/ch/0/biquad/0/repr/Filter/typ
/ch/0/biquad/0/repr/Filter/frequency
/ch/0/biquad/0/repr/Filter/gain
/ch/0/biquad/0/repr/Filter/shelf
/ch/0/biquad/0/repr/Filter/shape
/ch/0/biquad/0/repr/Filter/offset
/ch/0/biquad/0/repr/Filter/min
/ch/0/biquad/0/repr/Filter/max
/ch/0/run
/ch/0/source/signal
/ch/0/source/frequency
/ch/0/source/symmetry
/ch/0/source/amplitude
/ch/0/source/offset
/

In [114]:
from filter_design import StabilizerParameters

# Code to convert PID coeffs to iir filter parameters-------------------------------------------------------
def pid_coefficients(Kii, Ki, Kp, Kd, Kdd, sample_period):
    """Calculate PID IIR filter coefficients."""

    # Determine filter order
    if Kii != 0:
        assert (Kdd, Kd) == (0, 0), \
                "IIR filters I^2 and D or D^2 gain are unsupported"
        order = 2
    elif Ki != 0:
        assert Kdd == 0, \
                "IIR filters with I and D^2 gain are unsupported"
        order = 1
    else:
        order = 0

    kernels = [
        [1, 0, 0],
        [1, -1, 0],
        [1, -2, 1]
    ]

    gains = [Kii, Ki, Kp, Kd, Kdd]
    w = 2*np.pi*sample_period
    b = [sum(gains[2 - order + i] * w**(order - i) * kernels[i][j]
             for i in range(3)) for j in range(3)]

    # Normalization is redundant because a0 is 1 in all cases.
    a = kernels[order]
    assert a[0] == 1
    return b + [-ai for ai in a[1:]] #[b0, b1, b2, -a1, -a2] IIR coefficients to be programmed into a
#       Stabilizer IIR filter configuration.
sampling_frequency = 800000 #Hz
#-----------------------------------------------------------------------------------------------------------

In [197]:
P = 23 # proportional gain in dB
KP = 10**(P/20)
# KP = 0
KD = 0.000 # differential gain
KI = 2*np.pi*KP*60
# KI = 0

# PI controller ----------------------------
# Ku = 28.1838 # ultimate gain when you start to see oscillations
# Tu = 51*10**(-6) # time period of oscillations
# KP = 0.45*Ku
# KI = 0.54*Ku/Tu
#-----------------------------------------------------

# PID controller ----------------------------
# Ku = 28.1838 # ultimate gain when you start to see oscillations
# Tu = 51*10**(-6) # time period of oscillations
# KP = 0.6*Ku
# KI = 1.2*Ku/Tu
# KD = 0.075*Ku*Tu
#----------------------------------------------------- 

# coeffs = pid_coefficients(0, 0, 1, 0, 0, 1/sampling_frequency)
coeffs = pid_coefficients(0, KI, KP, KD, 0, 1/sampling_frequency)

V_setpoint = 195*10**(-3) # set point in volts (should be +ve)
b0 = coeffs[0]
b1 = coeffs[1]
b2 = coeffs[2]
a1 = -coeffs[3]
# print(a1)
# a1 = -1
a2 = -coeffs[4]
offset = -(32767/10)*(b2+b1+b0)*V_setpoint
# print(offset)

filter_data = {
    "ba":[b0,b1,b2,a1,a2],
    "u":offset,
    "min": 0, #-32767,
    "max": 32767
}

# Convert values to machine units
# y_offset = StabilizerParameters.volt_to_mu(filter_data["u"])
# print(y_offset)
y_min = filter_data["min"]
# print(y_min)
y_max = filter_data["max"]

# Prepare payload
# payload = {
#     "ba":filter_data["ba"],
#     "u":float(offset),
#     "min":float(y_min),
#     "max":float(y_max)
# }
payload = json.dumps(filter_data)
print(payload)
# Send to Stabilizer via MQTT
async def send_filter():
    # broker = "localhost"  # Change if needed
    # prefix = "dt/sinara/stabilizer/settings/iir_ch/0/0"
    # try:
    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)
        # print(await dev.get("/ch/0/biquad/0/repr/Raw"))
#         await dev.set("/ch/0/biquad/0/repr/Raw", {
#   "ba": [
#     14.167199006528879,
#     -14.12537544622754,
#     0,
#     -1,
#     0
#   ],
#   "u": -25.5009084608273197,
#   "min": -32767,
#   "max": 32767
# })
        await dev.set("/ch/0/biquad/0/repr/Raw", json.loads(json.dumps(filter_data)))
        print("Filter settings applied!")
    # except asyncio.TimeoutError:
    #     print("Error: MQTT request took too long!")
    # except Exception as e:
        # print(f"Unexpected error: {e}")

await send_filter()

{"ba": [14.167199006528879, -14.12537544622754, 0.0, -1, 0], "u": -26.723435707682256, "min": 0, "max": 32767}
Filter settings applied!


In [124]:
# test code
async def update_telemetry():
    # broker = "localhost"  # Change if needed
    # prefix = "dt/sinara/stabilizer/settings/iir_ch/0/0"
    # try:
    async with miniconf.Client(broker, protocol=miniconf.MQTTv5) as client:
        dev = miniconf.Miniconf(client, prefix)
        await dev.set("/telemetry_period", 10)
        print("Filter settings applied!")

await update_telemetry()

Filter settings applied!
